# Building MCP Servers for Compliance Documentation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ethanolivertroy/pcidss-mcp-training/blob/main/build-mcp-server.ipynb)

**A hands-on tutorial for GRC professionals**

---

## What You'll Learn

This notebook teaches you how to build a Model Context Protocol (MCP) server for compliance documentation. We use PCI-DSS as the example, but these patterns work for **any** compliance framework:

- NIST 800-53
- ISO 27001
- SOC 2
- FedRAMP
- HIPAA

### What is an MCP Server?

MCP servers provide **tools** that AI assistants like Claude can call to:
- Search structured compliance data
- Retrieve specific requirements by ID
- Map controls between frameworks
- Generate compliance checklists

### This Notebook is Fully Self-Contained

Everything runs right here - both Python AND TypeScript code. Works in:
- Google Colab
- Local Jupyter
- VS Code with Colab extension

---

## Part 1: Data Source - PCI-DSS Prioritized Approach Tool

### Why Excel Over PDF?

Many compliance documents are available as PDFs, but PDFs are difficult to parse. The same data is often available in Excel format, which is:

- **Already structured** - No complex extraction needed
- **Reliable** - Consistent column layouts
- **Fast** - Parse in milliseconds vs. minutes
- **Accurate** - No OCR errors or layout issues

For PCI-DSS, the **Prioritized Approach Tool** is an official Excel file from PCI Security Standards Council.

### Download the Excel File

1. Download from PCI SSC:
   
   **[Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx](https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx)**

2. Accept the license agreement

3. Upload when prompted (Colab) or place in your project directory (local)

### Legal Notes

The Excel file is licensed by PCI SSC for **internal use only**:
- Use for your organization's compliance work
- Do not redistribute converted data publicly
- Each user should download and accept the license directly

## Part 2: Parsing Excel Data

The Prioritized Approach Tool contains a sheet called "Prioritized Approach Milestones" with:

| Column A | Column B |
|----------|----------|
| Requirement text | Milestone priority (1-6) |

Requirements follow a hierarchical numbering:
- **Level 1**: "Requirement 1: Install and Maintain Network Security Controls"
- **Level 2**: "1.1 Processes and mechanisms..."
- **Level 3**: "1.1.1 All security policies..."

In [ ]:
# Install dependencies
!pip install -q openpyxl
print("Dependencies installed.")

In [ ]:
from pathlib import Path
import re
from typing import Dict, List, Any, Optional, Tuple

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally")

if IN_COLAB:
    from google.colab import files
    print("\nPlease upload the Prioritized Approach Tool Excel file...")
    print("Download from: https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    uploaded = files.upload()
    excel_path = Path(list(uploaded.keys())[0])
    print(f"\nUploaded: {excel_path.name}")
else:
    excel_path = Path("Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    if not excel_path.exists():
        print(f"Excel file not found. Please download from:")
        print("https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    else:
        print(f"Found: {excel_path.name}")

In [ ]:
def parse_requirement_cell(text: str) -> Optional[Dict[str, Any]]:
    """Parse requirement ID and title from Excel cell."""
    if not text:
        return None
    
    text = str(text).strip()
    
    # Pattern: "1.1.1 Title text"
    match = re.match(r'^(\d+(?:\.\d+)*)\s+(.+)', text)
    if match:
        req_id = match.group(1)
        title = match.group(2).strip()
        level = len(req_id.split('.'))
        return {'id': req_id, 'title': title, 'level': level}
    
    # Pattern: "Requirement 1: Title"
    section_match = re.match(r'^Requirement\s+(\d+):\s*(.+)', text, re.IGNORECASE)
    if section_match:
        return {
            'id': section_match.group(1),
            'title': section_match.group(2).strip(),
            'level': 1
        }
    
    return None

# Test
test_cases = [
    "Requirement 1: Install and Maintain Network Security Controls",
    "1.1 Processes and mechanisms for network security controls",
    "1.2.3 Specific requirement text"
]
print("Testing parser:")
for text in test_cases:
    result = parse_requirement_cell(text)
    if result:
        print(f"  {result['id']}: Level {result['level']}")

In [ ]:
from openpyxl import load_workbook
import warnings

# Suppress harmless openpyxl warning about Data Validation
# (Excel has dropdown menus that openpyxl ignores - doesn't affect our data)
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

def extract_requirements(excel_path: Path) -> List[Dict[str, Any]]:
    """Extract requirements from Excel file."""
    workbook = load_workbook(excel_path, data_only=True)
    
    sheet_name = "Prioritized Approach Milestones"
    if sheet_name not in workbook.sheetnames:
        print(f"Available sheets: {workbook.sheetnames}")
        return []
    
    worksheet = workbook[sheet_name]
    requirements = []
    
    for row in worksheet.iter_rows(min_row=3, values_only=True):
        if not row or not row[0]:
            continue
        
        parsed = parse_requirement_cell(str(row[0]))
        if not parsed:
            continue
        
        parts = parsed['id'].split('.')
        parent_id = '.'.join(parts[:-1]) if len(parts) > 1 else None
        
        requirement = {
            'id': parsed['id'],
            'level': parsed['level'],
            'parentId': parent_id,
            'title': parsed['title'],
            'statement': parsed['title'],
        }
        
        if row[1]:
            requirement['milestone'] = int(row[1])
        
        requirements.append(requirement)
    
    requirements.sort(key=lambda r: [int(x) for x in r['id'].split('.')])
    return requirements

# Extract requirements
if excel_path.exists():
    requirements = extract_requirements(excel_path)
    print(f"Extracted {len(requirements)} requirements")
    
    levels = {}
    for req in requirements:
        levels[req['level']] = levels.get(req['level'], 0) + 1
    
    # Show level distribution with explanation
    print("\nLevel distribution:")
    level_names = {
        1: "Principal requirements (Requirement 1-12)",
        2: "Sub-requirements (1.1, 1.2, etc.)",
        3: "Detailed requirements (1.1.1, 1.2.3, etc.)",
        4: "Specific controls (1.2.3.1, etc.)",
        5: "Deep nesting (rare)"
    }
    for level in sorted(levels.keys()):
        desc = level_names.get(level, "")
        print(f"  Level {level}: {levels[level]:>3}  {desc}")
    
    print(f"\nThis is the expected distribution for PCI-DSS v4.0.1")
else:
    requirements = []

## Part 3: Building a Fast Index

When an AI asks "What is requirement 1.2.3?", we need O(1) lookup, not O(n) linear search.

In [ ]:
class RequirementIndex:
    """Fast O(1) index for requirement lookups."""
    
    def __init__(self):
        self.requirements: Dict[str, Dict] = {}
        self.by_level: Dict[int, List[str]] = {}
        
    def add(self, req: Dict):
        self.requirements[req['id']] = req
        level = req['level']
        if level not in self.by_level:
            self.by_level[level] = []
        self.by_level[level].append(req['id'])
    
    def get(self, req_id: str) -> Optional[Dict]:
        return self.requirements.get(req_id)
    
    def get_children(self, parent_id: str) -> List[Dict]:
        return sorted(
            [r for r in self.requirements.values() if r.get('parentId') == parent_id],
            key=lambda r: [int(x) for x in r['id'].split('.')]
        )

# Build index
if requirements:
    index = RequirementIndex()
    for req in requirements:
        index.add(req)
    print(f"Indexed {len(index.requirements)} requirements")
    
    # Test
    req = index.get("1.2")
    if req:
        print(f"\nTest lookup 1.2: {req['title'][:50]}...")
        children = index.get_children("1.2")
        print(f"Children of 1.2: {len(children)}")

## Part 4: MCP Tool Patterns (Python)

Every MCP tool follows: **Schema -> Execute -> Return**

In [ ]:
from dataclasses import dataclass

@dataclass
class GetRequirementInput:
    id: str
    include_children: bool = False

def get_requirement_tool(input: GetRequirementInput, idx: RequirementIndex):
    """MCP tool pattern: validate -> execute -> return"""
    req = idx.get(input.id)
    if not req:
        raise ValueError(f"Requirement not found: {input.id}")
    
    result = {'requirement': req}
    if input.include_children:
        result['children'] = idx.get_children(input.id)
    return result

# Test
if requirements:
    result = get_requirement_tool(GetRequirementInput(id="1", include_children=True), index)
    print(f"Requirement 1: {result['requirement']['title'][:40]}...")
    print(f"Children: {len(result['children'])}")

## Part 5: Full-Text Search

TF-IDF scoring for "find requirements about encryption" queries.

In [ ]:
from collections import Counter
import math

class SearchIndex:
    """Simple TF-IDF search."""
    
    def __init__(self, reqs: List[Dict]):
        self.requirements = {r['id']: r for r in reqs}
        self.inverted_index: Dict[str, List] = {}
        self.doc_count = len(reqs)
        
        for req in reqs:
            text = f"{req['id']} {req['title']}".lower()
            word_counts = Counter(text.split())
            for word, count in word_counts.items():
                if word not in self.inverted_index:
                    self.inverted_index[word] = []
                self.inverted_index[word].append((req['id'], count))
    
    def search(self, query: str, limit: int = 5):
        scores = {}
        for word in query.lower().split():
            if word not in self.inverted_index:
                continue
            idf = math.log(self.doc_count / len(self.inverted_index[word]))
            for doc_id, tf in self.inverted_index[word]:
                scores[doc_id] = scores.get(doc_id, 0) + tf * idf
        
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:limit]

# Test
if requirements:
    search_index = SearchIndex(requirements)
    results = search_index.search("network security firewall")
    print("Search: 'network security firewall'")
    for req_id, score in results:
        print(f"  [{req_id}] {search_index.requirements[req_id]['title'][:50]}...")

## Part 6: Save Data as JSON

Export for use in the MCP server.

In [ ]:
import json
from datetime import datetime

if requirements:
    output = {
        'info': {
            'version': '4.0.1',
            'convertedAt': datetime.now().isoformat(),
        },
        'requirements': requirements,
    }
    
    output_file = Path('requirements.json')
    with open(output_file, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"Saved: {output_file} ({output_file.stat().st_size / 1024:.1f} KB)")
    print(f"Requirements: {len(requirements)}")
    
    if IN_COLAB:
        print("\n(File will be used by TypeScript server below)")

## Part 7: Adapting for Other Frameworks

The same patterns work for any compliance framework:

### NIST 800-53
```python
# Pattern: AC-1, AC-1(1)
match = re.match(r'^([A-Z]{2}-\d+(?:\(\d+\))?)\s+(.+)$', text)
```

### ISO 27001
```python
# Pattern: A.5.1.1
match = re.match(r'^(A\.\d+(?:\.\d+)*)\s+(.+)$', text)
```

### Key Steps
1. Find structured data (Excel/CSV > PDF)
2. Write regex for your ID format
3. Build the same index structure
4. Adapt tool descriptions

---

## Part 8: Building the MCP Server in TypeScript

Now let's build a **real, working MCP server** in TypeScript - right here in this notebook!

We'll use **Deno** - a TypeScript runtime that runs TS directly without compilation.

### Why Deno?
- Runs TypeScript directly (no build step)
- Easy to install anywhere
- Works in Colab, local Jupyter, VS Code
- Modern, secure runtime

In [ ]:
# Install Deno (works in Colab and locally)
import subprocess
import os

# Check if Deno is installed
deno_path = os.path.expanduser("~/.deno/bin/deno")
if not os.path.exists(deno_path):
    print("Installing Deno (one-time setup)...")
    subprocess.run("curl -fsSL https://deno.land/install.sh | sh", shell=True, capture_output=True)
    print("Deno installed!")
else:
    print("Deno already installed")

# Add to PATH for this session
os.environ["PATH"] = os.path.expanduser("~/.deno/bin") + ":" + os.environ["PATH"]

# Verify
!~/.deno/bin/deno --version

### Creating the MCP Server

The following cell writes a complete MCP server to a TypeScript file. This server:
1. Loads the `requirements.json` we created in Part 6
2. Provides `get_requirement`, `search_requirements`, and `list_requirements` tools
3. Uses the same indexing patterns we learned in Python

In [ ]:
%%writefile mcp_server.ts
// MCP Server for Compliance Documentation
// This file is generated by the notebook and can be run with Deno

// =============================================================================
// Types
// =============================================================================

interface Requirement {
  id: string;
  level: number;
  parentId: string | null;
  title: string;
  statement: string;
  milestone?: number;
}

interface RequirementData {
  info: { version: string };
  requirements: Requirement[];
}

// =============================================================================
// Index - O(1) Lookups (same pattern as Python!)
// =============================================================================

class RequirementIndex {
  private byId = new Map<string, Requirement>();
  private byLevel = new Map<number, string[]>();

  constructor(requirements: Requirement[]) {
    for (const req of requirements) {
      this.byId.set(req.id, req);
      const levelList = this.byLevel.get(req.level) || [];
      levelList.push(req.id);
      this.byLevel.set(req.level, levelList);
    }
  }

  get(id: string): Requirement | undefined {
    return this.byId.get(id);
  }

  getByLevel(level: number): Requirement[] {
    const ids = this.byLevel.get(level) || [];
    return ids.map(id => this.byId.get(id)!).filter(Boolean);
  }

  getChildren(parentId: string): Requirement[] {
    return Array.from(this.byId.values())
      .filter(r => r.parentId === parentId)
      .sort((a, b) => a.id.localeCompare(b.id, undefined, { numeric: true }));
  }

  search(query: string, limit = 10): Requirement[] {
    const terms = query.toLowerCase().split(/\s+/);
    const scores = new Map<string, number>();

    for (const [id, req] of this.byId) {
      const text = `${req.id} ${req.title} ${req.statement}`.toLowerCase();
      let score = 0;
      for (const term of terms) {
        if (text.includes(term)) score++;
      }
      if (score > 0) scores.set(id, score);
    }

    return Array.from(scores.entries())
      .sort((a, b) => b[1] - a[1])
      .slice(0, limit)
      .map(([id]) => this.byId.get(id)!);
  }

  all(): Requirement[] {
    return Array.from(this.byId.values());
  }
}

// =============================================================================
// Tool Functions (simulating MCP tools)
// =============================================================================

function getRequirement(index: RequirementIndex, id: string, includeChildren = false) {
  const requirement = index.get(id);
  if (!requirement) {
    return { error: `Requirement not found: ${id}` };
  }
  
  const result: any = { requirement };
  if (includeChildren) {
    result.children = index.getChildren(id);
  }
  return result;
}

function searchRequirements(index: RequirementIndex, query: string, limit = 10) {
  const results = index.search(query, limit);
  return {
    query,
    count: results.length,
    results: results.map(r => ({ id: r.id, title: r.title }))
  };
}

function listRequirements(index: RequirementIndex, level?: number, limit = 20) {
  let results = level ? index.getByLevel(level) : index.all();
  results = results.slice(0, limit);
  return {
    level: level || "all",
    count: results.length,
    requirements: results.map(r => ({ id: r.id, title: r.title }))
  };
}

// =============================================================================
// Main - Demo the server functionality
// =============================================================================

async function main() {
  // Load requirements from JSON
  const data: RequirementData = JSON.parse(
    await Deno.readTextFile("requirements.json")
  );
  
  const index = new RequirementIndex(data.requirements);
  console.log(`\n=== MCP Server Demo ===");
  console.log(`Loaded ${index.all().length} requirements from PCI-DSS v${data.info.version}\n`);

  // Demo: get_requirement tool
  console.log("--- Tool: get_requirement ---");
  const req1 = getRequirement(index, "1", true);
  console.log(`Requirement 1: ${req1.requirement.title}`);
  console.log(`Children: ${req1.children.length} sub-requirements\n`);

  // Demo: search_requirements tool
  console.log("--- Tool: search_requirements ---");
  const search1 = searchRequirements(index, "firewall network", 3);
  console.log(`Search "firewall network": ${search1.count} results`);
  for (const r of search1.results) {
    console.log(`  [${r.id}] ${r.title.slice(0, 50)}...`);
  }
  console.log();

  // Demo: list_requirements tool
  console.log("--- Tool: list_requirements ---");
  const list1 = listRequirements(index, 1);
  console.log(`Level 1 requirements (principal): ${list1.count}`);
  for (const r of list1.requirements.slice(0, 3)) {
    console.log(`  [${r.id}] ${r.title.slice(0, 50)}...`);
  }
  console.log("  ...");

  console.log("\n=== Server Demo Complete ===");
  console.log("This same code can be adapted to run as a real MCP server!");
}

main();

In [ ]:
# Run the TypeScript MCP server demo!
!~/.deno/bin/deno run --allow-read mcp_server.ts

### It Works!

You just ran a TypeScript MCP server that:
1. Loaded the `requirements.json` from Part 6
2. Built an O(1) index (same pattern as Python)
3. Executed the same tools an AI assistant would call

---

## Deploying as a Real MCP Server

To use this with Claude Desktop, you need to add the MCP SDK. Here's the production version:

In [ ]:
%%writefile mcp_server_full.ts
// Full MCP Server with SDK
// Run with: deno run --allow-read --allow-net mcp_server_full.ts

// For a real deployment, you would:
// 1. Use Node.js with @modelcontextprotocol/sdk
// 2. Or use Deno with the MCP SDK from npm

/*
// Node.js version (package.json):
{
  "dependencies": {
    "@modelcontextprotocol/sdk": "^1.0.0",
    "zod": "^3.22.0"
  }
}

// Then in your index.ts:
import { McpServer } from "@modelcontextprotocol/sdk/server/mcp.js";
import { StdioServerTransport } from "@modelcontextprotocol/sdk/server/stdio.js";
import { z } from "zod";

const server = new McpServer({ name: "compliance-server", version: "1.0.0" });

server.tool(
  "get_requirement",
  "Get a specific requirement by ID",
  { id: z.string(), includeChildren: z.boolean().default(false) },
  async ({ id, includeChildren }) => {
    // Your implementation here
  }
);

const transport = new StdioServerTransport();
await server.connect(transport);
*/

console.log("See the comments above for production MCP server setup!");
console.log("\nFor Claude Desktop, add to ~/.claude/claude_desktop_config.json:");
console.log(`
{
  "mcpServers": {
    "compliance": {
      "command": "node",
      "args": ["/path/to/your/dist/index.js"]
    }
  }
}
`);

In [ ]:
!~/.deno/bin/deno run mcp_server_full.ts

---

## Summary

You've learned to build MCP servers for compliance documentation:

1. **Data Extraction** - Excel parsing with openpyxl
2. **Indexing** - O(1) Map lookups (Python and TypeScript)
3. **MCP Tool Patterns** - Schema -> Execute -> Return
4. **Full-Text Search** - TF-IDF scoring
5. **Working Server** - TypeScript code that runs right here!
6. **Adaptation** - Same patterns for any framework

### Resources

- [MCP Specification](https://modelcontextprotocol.io/specification/2025-11-25)
- [MCP TypeScript SDK](https://github.com/modelcontextprotocol/typescript-sdk)
- [Deno Runtime](https://deno.land)
- [This Tutorial](https://github.com/ethanolivertroy/pcidss-mcp-training)

---

**Go build something!**